In [1]:
#links uteis:
#https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.Connection
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html

# Importando Bibliotecas 

In [2]:
#importando bibliotecas
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import numpy as np
#from google.cloud import bigquery

# Criando conexão com o Banco para Extração

In [3]:
# Dados da conexão
server = 'DESKTOP-LD5D6LA'
database = 'MestreElectronics'
driver = 'SQL Server'
conn_str = f'mssql+pyodbc://{server}/{database}?driver={driver}'

# Criar uma conexão SQLAlchemy
conexao = create_engine(conn_str)

# Lendo os dados

In [4]:
#Lendo as Tabelas e colocando em DataFrames
produtos_df = pd.read_sql('SELECT * FROM Product',conexao)
prod_category_df = pd.read_sql('SELECT * FROM ProductCategory',conexao)
prod_subcategory_df = pd.read_sql('SELECT * FROM ProductSubcategory',conexao)

cliente_df = pd.read_sql('SELECT * FROM Customer',conexao)
localidade_df = pd.read_sql('SELECT * FROM Geography',conexao)

vendas_df = pd.read_sql('SELECT * FROM Sales',conexao)
vendas_on_df = pd.read_sql('SELECT * FROM OnlineSales',conexao)

lojas_df = pd.read_sql('SELECT * FROM Store',conexao)
estoque_df = pd.read_sql('SELECT * FROM Inventory',conexao)
funcionarios_df = pd.read_sql('SELECT * FROM Employee',conexao)

canal_df = pd.read_sql('SELECT * FROM Channel',conexao)

# Dimensão Produto

In [5]:
#DIM_PRODUTOS
produtos_df = produtos_df.merge(prod_subcategory_df)

In [6]:
produtos_df = produtos_df.merge(prod_category_df)

In [7]:
#Excluir Colunas
#colunas_para_excluir = ['ProductDescription', 'ColorName']
#produtos_df = produtos_df.drop(columns= colunas_para_excluir)

In [8]:
#Manter Colunas
produtos_df = produtos_df[['ProductKey','ProductName','BrandName','ClassName','ProductSubcategoryName','ProductCategoryName']]


In [9]:
#DIM_PRODUTOS
#display(produtos_df.head(3))

# Dimensão Cliente

In [10]:
#DIM_CLIENTE
cliente_df = cliente_df.merge(localidade_df)

In [11]:
#Concatenando Colunas
cliente_df['FullName'] = cliente_df['FirstName'] + ' ' + cliente_df['LastName']

In [12]:
#Convertendo coluna para o Formato de Data
cliente_df['BirthDate'] = pd.to_datetime(cliente_df['BirthDate'])

In [13]:
#Função para calcular a idade
def calcular_idade(BirthDate):
    hoje = datetime.today()
    return hoje.year - BirthDate.year - ((hoje.month, hoje.day) < (BirthDate.month, BirthDate.day))

In [14]:
#Aplicando a Função ao Dataframe e Criando a coluna de idade
cliente_df['Idade'] = cliente_df['BirthDate'].apply(calcular_idade)

In [15]:
#Criando uma nova coluna com condição
cliente_df['CustomerName'] = np.where(cliente_df['CustomerType'] == 'Person', cliente_df['FullName'], cliente_df['CompanyName'])

In [16]:
#Excluir Colunas
colunas_para_excluir = ['GeographyKey', 'FirstName', 'LastName', 'BirthDate', 'CompanyName', 'GeographyType','StateProvinceName','FullName']
cliente_df = cliente_df.drop(columns= colunas_para_excluir)

In [17]:
#DIM_CLIENTE
#display(cliente_df.head(4))

# Fato Vendas

In [18]:
#FT_VENDAS
#Converter os Tipos de Dados e mesclando
canal_df['ChannelLabel'] = canal_df['ChannelLabel'].astype(int)

vendas_df = vendas_df.merge(canal_df, left_on = 'channelKey', right_on = 'ChannelLabel')

In [19]:
vendas_df = vendas_df.drop(columns = ['UpdateDate','LoadDate','ETLLoadID','ChannelLabel','ChannelDescription','ChannelKey','channelKey'])

In [20]:
#FT_VENDAS
#display(vendas_df.head(5))

# Fato Vendas Online

In [21]:
#FT_VENDAS_ONLINE
display(vendas_on_df.head(2))

,OnlineSalesKey,DateOrder,DateShip,ProductKey,StoreKey,CustomerKey,SalesOrderNumber,UnitCost,UnitPrice,SalesQuantity,DiscountAmount,SalesAmount,TotalCost
0,20832,2017-07-09,2017-07-22,8,199,3089,20070709214088,30.58,59.99,1,6.0,53.99,30.58
1,20833,2017-07-09,2017-07-22,684,199,18789,200707093CS466,73.58,160.00,7,112.0,1008.00,515.06


# Dimensão Loja

In [22]:
#DIM_LOJA
lojas_df = lojas_df.merge(localidade_df)


In [23]:
#DIM_LOJA
#Manter Coluna
lojas_df =lojas_df[['StoreKey','StoreManager','StoreType','StoreName','EmployeeCount','ContinentName','CityName','RegionCountryName']]
#display(lojas_df.head(5))

# Dimensão Funcionario

In [24]:
#DIM_FUNCIONARIO
#Concatenando colunas
funcionarios_df['FullName'] = funcionarios_df['FirstName'] + ' ' + funcionarios_df['LastName']

In [25]:
#Convertendo Dados
funcionarios_df['StartDate'] = pd.to_datetime(funcionarios_df['StartDate'])
funcionarios_df['EndDate'] = pd.to_datetime(funcionarios_df['EndDate'])

In [26]:
#Substituindo valores NULO pela data Atual na Colua de EndDat
funcionarios_df['EndDate'] = funcionarios_df['EndDate'].fillna(pd.Timestamp('today')).dt.normalize()

In [27]:
# Calculando a duração
funcionarios_df['Duration'] = funcionarios_df['EndDate'] - funcionarios_df['StartDate']
funcionarios_df['Duration_in_Days'] = funcionarios_df['Duration'].dt.days

In [28]:
funcionarios_df = funcionarios_df.drop(columns = ['FirstName','LastName','Duration'])

In [29]:
#display(funcionarios_df.head(4))

# Fato Estoque

In [30]:
display(estoque_df.head(2))

,InventoryKey,DateKey,StoreKey,ProductKey,OnHandQuantity,OnOrderQuantity,SafetyStockQuantity,UnitCost,DaysInStock,MinDayInStock,MaxDayInStock
0,56961,2017-05-19,294,1141,9,0,3,144.52,8,33,85
1,56962,2017-06-16,53,1530,10,0,18,122.32,11,7,60


# Conectando no Dados no Banco que vamos Inserir os Dados

In [31]:
# Criar banco de dados (se necessário)
conexao.execute("IF NOT EXISTS(SELECT * FROM sys.databases WHERE name='DataWarehouse') CREATE DATABASE DataWarehouse")


In [32]:
# Conectando ao banco de Dados da Inserção
server = 'DESKTOP-LD5D6LA'
database = 'DataWarehouse'
driver = 'SQL Server'
conn_str_dw = f'mssql+pyodbc://{server}/{database}?driver={driver}'

# Criar uma conexão SQLAlchemy
conexao_dw  = create_engine(conn_str_dw)

# Inserindo os Dados

In [33]:
#Inserindo dados de Produtos
produtos_df.to_sql('DIM_Produto', con=conexao_dw, index=False, if_exists='replace')

-1

In [34]:
#Inserindo dados de Clientes
cliente_df.to_sql('DIM_Cliente', con=conexao_dw, index=False, if_exists='replace')

-1

In [35]:
#Inserindo dados de Lojas
lojas_df.to_sql('DIM_Lojas', con=conexao_dw, index=False, if_exists='replace')

-1

In [36]:
#Inserindo dados de Funcionarios
funcionarios_df.to_sql('DIM_Funcionarios', con=conexao_dw, index=False, if_exists='replace')

-1

In [37]:
#Inserindo dados de Vendas
vendas_df.to_sql('FT_Vendas', con=conexao_dw, index=False, if_exists='replace')

-1

In [38]:
#Inserindo dados de Vendas_Online
vendas_on_df.to_sql('FT_Vendas_Online', con=conexao_dw, index=False, if_exists='replace')

-1

In [39]:
#Inserindo dados de Estoque
estoque_df.to_sql('FT_Estoque', con=conexao_dw, index=False, if_exists='replace')

-1